In [9]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('/home/gafur/Documents/Finance/Zillow_NLP/data/processed_zillow_reviews.csv') 

#I'll sample proportionally based on existing data distribution
target_size = 100

sampled_df = df.groupby('Rating').apply(
    lambda x: x.sample(frac=target_size / len(df), random_state=42)
).reset_index(drop=True)

print(sampled_df['Rating'].value_counts())
sampled_df

/tmp/ipykernel_82371/567229453.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('Rating').apply(


Rating
5    99
4     1
Name: count, dtype: int64


,Rating,Address,ReviewDate,Description,Cleaned_Description
0,4,"5420 La Jolla Blvd #B204, La Jolla, CA 92037",2020-08-06,"Linda is very knowledgeable and experienced. Despite the fact that I had bought many homes in the past, she guided me every step of the way. Her follow up was impeccable. She is a consummate professional. Always going above and beyond the call of duty. I would highly recommend anyone looking for an exceptional realtor. And this review doesn’t even begin to do her justice.",linda knowledgeable experienced despite fact buy home past guide step way follow impeccable consummate professional go duty highly recommend look exceptional realtor review begin justice
1,5,"10123 Rio Hondo Pkwy, El Monte, CA 91733",2012-07-22,"Our family has worked with Victor for over a year he represented in the purchase of a rental property. Victor advised us to submit an fair market price which was accepted by the seller. Once escrow opened, Victor stayed on top of the Escrow company to facilitate any issues and in some cases doing that the sales agent should have done.<br/><br/>We are still working with Victor to purchase another property. Victor monitors listings that meet our criteria and sends update of the latest listings in the areas where we are interested. <br/><br/>He is very knowledgeable of the areas and has an excellent sense of market values. He is also aware of our specific criteria and provides us frequent updates on the market.",family work victor year represent purchase rental property victor advise submit fair market price accept seller escrow open victor stay escrow company facilitate issue case sale agent work victor purchase property victor monitor listing meet criterion send update late listing area interested knowledgeable area excellent sense market value aware specific criterion provide frequent update market
2,5,"1314 Brown St, Martinez, CA 94553",2020-08-23,"Cynthia blew us away with how helpful, patient and knowledgeable she was while we were searching for our first home to purchase. She was in constant communication and helped us find and close on our dream home during the busy holiday season. The amount of paperwork and contracts can be very daunting for a first time buyer but she came to us and walked us through everything step by step, even bringing gifts to help entertain our toddler during our meetings. We had such a great experience and are so happy with our decision to work with Cynthia. We highly recommend her to anyone in need of buying or selling a home. Thank you Cynthia!",cynthia blow away helpful patient knowledgeable search home purchase constant communication help find close dream home busy holiday season paperwork contract daunting time buyer come walk step step bring gift help entertain toddler meeting great experience happy decision work cynthia highly recommend need buy sell home thank cynthia
3,5,"17975 Climax Rd Jackson, CA 95642",2023-10-06,"Janelle is an outstanding realtor. She was present and available throughout the entire process. Her negotiation skills are creative and top notch. Janelle was in constant contact with the seller, our mortgage broker, the title company, and inspectors. She made everything run smoothly. We are a family of three with one more on the way and we are so excited to be moving into our new home. From finding a home that was not even on the market, to closing in under thirty days. Thank you Janelle, we could not have done this without you.",janelle outstanding realtor present available entire process negotiation skill creative notch janelle constant contact seller mortgage broker title company inspector run smoothly family way excited move new home find home market close thirty day thank janelle
4,5,"1945 W 147th St Gardena, CA 90249",2022-11-11,"Katelyn is AWESOME! She helped us buy our first house when the market was crazy. She guided us through the process with grace and ease. Not only did she really ground us during this crazy process, she was such a 

In [10]:
from pyabsa import ATEPCCheckpointManager

# Download and load pretrained ATEPC model
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(
    checkpoint='english',  # You can also try 'english' or 'fast_lcf_atepc'
    auto_device=True  # uses GPU if available
)


[2025-03-25 13:41:29] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-03-25 13:41:29] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-03-25 13:41:29] (2.4.1.post1) Downloading checkpoint:english 
[2025-03-25 13:41:29] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-03-25 13:41:29] (2.4.1.post1) Checkpoint already downloaded, skip
[2025-03-25 13:41:29] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2025-03-25 13:41:29] (2.4.1.post1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2025-03-25 13:41:29] (2.4.1.post1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apc

/home/gafur/Documents/Finance/Zillow_NLP/venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
# Run ABSA on each cleaned review
def extract_absa(text):
    result = aspect_extractor.extract_aspect([text], print_result=False, save_result=False)
    return result[0] if result else None

# Apply to the Cleaned_Description column
sampled_df['ABSA_Result'] = sampled_df['Cleaned_Description'].apply(extract_absa)


In [12]:
def parse_absa_result(res):
    if res and 'aspect' in res:
        return ', '.join([f"{asp} ({sent})" for asp, sent in zip(res['aspect'], res['sentiment'])])
    return None

sampled_df['Aspect_Sentiment_Pairs'] = sampled_df['ABSA_Result'].apply(parse_absa_result)


In [13]:
sampled_df['Aspect_Sentiment_Pairs']

0            linda (Positive)
1                            
2                            
3          janelle (Positive)
4          katelyn (Positive)
               ...           
95        jennifer (Positive)
96            nika (Positive)
97                           
98    service look (Positive)
99         theresa (Positive)
Name: Aspect_Sentiment_Pairs, Length: 100, dtype: object

In [14]:
sampled_df.to_csv("/home/gafur/Documents/Finance/Zillow_NLP/data/absa_on_100_review.csv")